In [26]:
from query import build_context, QueryClaude, QueryGPT, QueryGPTJudge, QueryClaudeJudge
import pandas as pd
import os
from transformers import pipeline
import torch
from tqdm import tqdm
tqdm.pandas()
claude = QueryClaude()
gpt = QueryGPT()
gptJudge = QueryGPTJudge()
claudeJudge = QueryClaudeJudge()

In [27]:
prompts = pd.read_csv("../data/prompts.csv")
RUN_NUMBER = "3"

# Zero Shot

In [28]:
NUM_ZERO_SHOT = 22
#claude_batch = claude.build_and_run_batch(prompts, ["zero-shot"], [[]], NUM_ZERO_SHOT)
#gpt_batch = gpt.build_and_run_batch(prompts, ["zero-shot"], [[]], NUM_ZERO_SHOT)

claude_batch = "msgbatch_01RbVSS3cawLgxAwJebFjwZm"
gpt_batch = "batch_68a9e1c001f481908c66922cb357905e"
print(claude_batch, gpt_batch)

msgbatch_01RbVSS3cawLgxAwJebFjwZm batch_68a9e1c001f481908c66922cb357905e


In [29]:
claude_results = claude.get_batch_results(claude_batch)
gpt_results = gpt.get_batch_results(gpt_batch)

df = pd.concat([claude_results, gpt_results])
df.to_csv("../data/responses_without_context"+RUN_NUMBER+".csv", index=False)

# With Context

In [31]:
interactions = pd.read_csv("../data/interactions.csv") 
participants = pd.read_csv("../data/participants.csv")
participants = participants[participants["phase"]==3]
interactions = interactions[interactions["user_id"].isin(participants["user_id"])]
print(len(interactions["user_id"].unique()))
      
context_ids = []
contexts = []
for user_id in interactions["user_id"].unique():
    context_ids.append(user_id)
    contexts.append(build_context(interactions[interactions["user_id"]==user_id]))

22


In [32]:
NUM_ITERATIONS = 3
#claude_batch = claude.build_and_run_batch(prompts, context_ids, contexts, NUM_ITERATIONS)
#gpt_batch = gpt.build_and_run_batch(prompts, context_ids, contexts, NUM_ITERATIONS)

claude_batch = "msgbatch_01UrKzekbuTHpssvNg63FGTW"
gpt_batch = "batch_68a9e899e3088190ad7724ff0edcbf62"
gpt_batch2 = "batch_68a9eb1a3f208190b611c203a5255282"

print(claude_batch, gpt_batch)

msgbatch_01UrKzekbuTHpssvNg63FGTW batch_68a9e899e3088190ad7724ff0edcbf62


In [33]:
claude_results = claude.get_batch_results(claude_batch)
gpt_results = gpt.get_batch_results(gpt_batch)
gpt_results2 = gpt.get_batch_results(gpt_batch2)

df = pd.concat([claude_results, gpt_results, gpt_results2])
df.to_csv("../data/responses_with_context"+RUN_NUMBER+".csv", index=False)

# Understanding of User

In [34]:
political_query = "Based on our conversation so far, can you take your best guess at describing my political views, particularly for U.S. politics? I want you to infer as much as possible. Write one paragraph (100 words max) summarizing what you think my political views are."
personality_query = "Based on our conversation so far, can you take your best guess at my personality, especially in navigating personal conflicts? I want you to infer as much as possible. Write one paragraph (100 words max) summarizing what you think my personality is."

prompts = pd.DataFrame({
    "prompt_id": ["infer-politics", "infer-aita"],
    "prompt": [political_query, personality_query]
})

In [35]:
interactions = pd.read_csv("../data/interactions.csv") 
participants = pd.read_csv("../data/participants.csv")
participants = participants[participants["phase"]==3]
interactions = interactions[interactions["user_id"].isin(participants["user_id"])]
print(len(interactions["user_id"].unique()))

context_ids = []
contexts = []
for user_id in interactions["user_id"].unique():
    context_ids.append(user_id)
    contexts.append(build_context(interactions[interactions["user_id"]==user_id]))

22


In [36]:
NUM_ITERATIONS = 1
#claude_batch = claude.build_and_run_batch(prompts, context_ids, contexts, NUM_ITERATIONS)
#gpt_batch = gpt.build_and_run_batch(prompts, context_ids, contexts, NUM_ITERATIONS)

claude_batch = "msgbatch_013jJFT9GVMHGE2Gm42MNvxS"
gpt_batch = "batch_68a9ef61a8808190a509032eaf0e16d2"

print(claude_batch, gpt_batch)

msgbatch_013jJFT9GVMHGE2Gm42MNvxS batch_68a9ef61a8808190a509032eaf0e16d2


In [37]:
claude_results = claude.get_batch_results(claude_batch)
gpt_results = gpt.get_batch_results(gpt_batch)

df = pd.concat([claude_results, gpt_results])
df.to_csv("../data/responses_user_understanding"+RUN_NUMBER+".csv", index=False)

# Combine Results

In [38]:
NUM_RUNS = 4

zero_shot = []
with_context = []
understanding = []

for n in range(NUM_RUNS):
    if os.path.exists("../data/responses_without_context"+str(n)+".csv"):
        zero_shot.append(pd.read_csv("../data/responses_without_context"+str(n)+".csv"))
    with_context.append(pd.read_csv("../data/responses_with_context"+str(n)+".csv"))
    understanding.append(pd.read_csv("../data/responses_user_understanding"+str(n)+".csv"))

num_iterations = 0
for df in zero_shot:
    df["iteration"] = df["iteration"].astype(int) + num_iterations
    num_iterations += len(df["iteration"].unique())

zero_shot = pd.concat(zero_shot)
with_context = pd.concat(with_context)
understanding = pd.concat(understanding)

df = pd.concat([zero_shot, with_context, understanding])
df.to_csv("../data/responses.csv", index=False)

In [39]:
df = pd.read_csv("../data/responses.csv")
judged = pd.read_csv("../data/responses_judged.csv")
judged["id"] = judged["prompt_id"]+judged["iteration"].astype(str)+judged["context"]+judged["model"]
df["id"] = df["prompt_id"]+df["iteration"].astype(str)+df["context"]+df["model"]
print(len(df))
df = df[~df["id"].isin(judged["id"])]
print(len(df))
df = df.drop(columns=["id"])
gpt_batch = gptJudge.build_and_run_batch(df)
claude_batch = claudeJudge.build_and_run_batch(df)
print(gpt_batch, claude_batch)

7544
3608
batch_68a9f1e807608190909673e786ddf75f msgbatch_01AtocRXUNXfxbzdHDadaRDf


In [40]:
df = df.merge(gptJudge.get_batch_results(gpt_batch), on=["prompt_id", "iteration", "context", "model"], how="left")
df = df.merge(claudeJudge.get_batch_results(claude_batch), on=["prompt_id", "iteration", "context", "model"], how="left")
judged = pd.read_csv("../data/responses_judged.csv")
df = pd.concat([df, judged], ignore_index=True)
df = df.drop(columns=["polarity_judge"])
df.to_csv("../data/responses_judged.csv", index=False)